#### Step 3: Model Fine-tuning
In this notebook, you'll fine-tune the Meta Llama 2 7B large language model, deploy the fine-tuned model, and test it's text generation and domain knowledge capabilities. 

Fine-tuning refers to the process of taking a pre-trained language model and retraining it for a different but related task using specific data. This approach is also known as transfer learning, which involves transferring the knowledge learned from one task to another. Large language models (LLMs) like Llama 2 7B are trained on massive amounts of unlabeled data and can be fine-tuned on domain domain datasets, making the model perform better on that specific domain.

Input: A train and an optional validation directory. Each directory contains a CSV/JSON/TXT file.
For CSV/JSON files, the train or validation data is used from the column called 'text' or the first column if no column called 'text' is found.
The number of files under train and validation should equal to one.

- **You'll choose your dataset below based on the domain you've chosen**

Output: A trained model that can be deployed for inference.\
After you've fine-tuned the model, you'll evaluate it with the same input you used in project step 2: model evaluation. 

---

#### Set up

---
Install and import the necessary packages. Restart the kernel after executing the cell below. 

---

In [1]:
!pip install --upgrade sagemaker datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.0/419.0 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.0
    Uninstalling fsspec-2024.6.0:
      Successfully uninstalled fsspec-2024.6.0
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.224.4
    Uninstalling sagemaker-2.224.4:
      Successfully uninstalled sagemaker-2.224.4
Select the model to fine-tune


Select the model to fine-tune

In [3]:
model_id, model_version = "meta-textgeneration-llama-2-7b", "2.*"

In the cell below, choose the training dataset text for the domain you've chosen and update the code in the cell below:  

To create a finance domain expert model: 

- `"training": f"s3://genaiwithawsproject2024/training-datasets/finance"`

To create a medical domain expert model: 

- `"training": f"s3://genaiwithawsproject2024/training-datasets/medical"`

To create an IT domain expert model: 

- `"training": f"s3://genaiwithawsproject2024/training-datasets/it"`

In [5]:
from sagemaker.jumpstart.estimator import JumpStartEstimator
from sagemaker.inputs import TrainingInput

# Define the model ID and instance type
model_id = 'meta-textgeneration-llama-2-7b'  # Replace with your model ID if different

# Create an instance of JumpStartEstimator
estimator = JumpStartEstimator(
    model_id=model_id,
    environment={"accept_eula": "true"},
    instance_type="ml.g5.2xlarge"
)

# Set hyperparameters for the training job
estimator.set_hyperparameters(
    instruction_tuned="False",
    epoch="5"
)

# Specify the S3 path to your text file using TrainingInput
training_data = TrainingInput(
    s3_data="s3://genaiwithaws-financialbucket/financialDataset.txt",
    content_type="text/plain"
)

# Start the training job with the specified S3 path
estimator.fit({"training": training_data})

sagemaker.config INFO - Not applying SDK defaults from location: arn:aws:iam::427537235262:role/service-role/SageMaker-genaiwithawsudacitySagemakerRole
2024-07-23 06:01:32 Starting - Starting the training job...
2024-07-23 06:01:50 Starting - Preparing the instances for training...
2024-07-23 06:02:23 Downloading - Downloading input 
2024-07-23 06:27:59 Training Complete


#### Deploy the fine-tuned model
---
Next, we deploy the domain fine-tuned model. We will compare the performance of the fine-tuned and pre-trained model.

---

In [7]:
finetuned_predictor = estimator.deploy()

 Endpoint created User: arn:aws:sts::427537235262:assumed-role/SageMaker-genaiwithawsudacitySagemakerRole/SageMaker


#### Evaluate the pre-trained and fine-tuned model
---
Next, we use the same input from the model evaluation step to evaluate the performance of the fine-tuned model and compare it with the base pre-trained model. 

---

Create a function to print the response from the model

In [9]:
def print_response(payload, response):
    print(f" {response}")
    print("\n==================================\n")

Now we can run the same prompts on the fine-tuned model to evaluate it's domain knowledge.   

**Replace "inputs"** in the next cell with the input to send the model based on the domain you've chosen. 

**For financial domain:**

  "inputs": "Replace with sentence below from text"  
- "The  investment  tests  performed  indicate"
- "the  relative  volume  for  the  long  out  of  the  money  options, indicates"
- "The  results  for  the  short  in  the  money  options"
- "The  results  are  encouraging  for  aggressive  investors"

**For medical domain:** 

  "inputs": "Replace with sentence below from text" 
- "Myeloid neoplasms and acute leukemias derive from"
- "Genomic characterization is essential for"
- "Certain germline disorders may be associated with"
- "In contrast to targeted approaches, genome-wide sequencing"

**For IT domain:** 

  "inputs": "Replace with sentence below from text" 
- "Traditional approaches to data management such as"
- "A second important aspect of ubiquitous computing environments is"
- "because ubiquitous computing is intended to" 
- "outline the key aspects of ubiquitous computing from a data management perspective."

In [11]:
payload = {
    "inputs": "The results are encouraging for aggressive investors",
    "parameters": {
        "max_new_tokens": 64,
        "top_p": 0.9,
        "temperature": 0.6,
        "return_full_text": False,
    },
}
try:
    response = finetuned_predictor.predict(payload, custom_attributes="accept_eula=true")
    print_response(payload, response)
except Exception as e:
    print(e)

The results are encouraging for aggressive investors with the opportunity to learn of split announcements early and acquire option positions quickly. Apparently the downside risk is minimal and the short-run returns can be quite good. For investors who can buy during day -1, the mean of the range generally exceeded a 10 percent return in four days. However, this might be considered evidence against the semi-strong efficient market hypothesis due to the information requirements and time constraints involved.


Do the outputs from the fine-tuned model provide domain-specific insightful and relevant content? You can continue experimenting with the inputs of the model to test it's domain knowledge. 

**Use the output from this notebook to fill out the "model fine-tuning" section of the project documentation report**

**After you've filled out the report, run the cells below to delete the model deployment** 

`IF YOU FAIL TO RUN THE CELLS BELOW YOU WILL RUN OUT OF BUDGET TO COMPLETE THE PROJECT`

In [14]:
finetuned_predictor.delete_model()
finetuned_predictor.delete_endpoint()

In [16]:
#Completed - Daksh Verma